In [ ]:
from my_obsidian_reader import ObsidianReader
from moc import build_notes_map, get_moc_and_linked_notes
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

In [ ]:
reader = ObsidianReader("/Users/chris/zk-copy-for-testing/", extract_tasks=True, remove_tasks_from_text=True)
documents = reader.load_data()
# Build a hash map keyed by file name.
notes_map = build_notes_map(documents)

In [ ]:
len(documents)
len(notes_map.keys())

In [ ]:
# Build an index for an MOC file
moc_file = "Post - Compound Knowledge in the Information Age.md"
moc_file = "MOC Idea - ASH - Augment stay human map.md"
moc_docs = get_moc_and_linked_notes(moc_file, notes_map, depth=2)
len(moc_docs)

In [ ]:
import pyperclip

def concatenate_documents_text(documents) -> str:
    text = "\n\n".join(doc.text for doc in documents)
    pyperclip.copy(text)
    print("Concatenated text has been copied to your clipboard.")

concatenate_documents_text(moc_docs)


In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# Build an index on the combined documents.
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
    device="cpu"  # Use "cuda" for GPU acceleration
)
index = VectorStoreIndex.from_documents(moc_docs)

In [ ]:
index.storage_context.persist(persist_dir="./storage")

In [ ]:
import os
load_dotenv("/Users/chris/repos/deep-learning/keys.env")

from llama_index.llms.anthropic import Anthropic

model_name = "claude-3-5-sonnet-20241022"
Settings.llm = Anthropic(model=model_name, max_tokens=4000)
query_engine = index.as_query_engine(similarity_top_k=25, response_mode="tree_summarize",)

In [ ]:
response = query_engine.query(
    "Based on all the notes you have related to 'Compound Knowledge in the Information Age', what are the key points? What am I trying to say? help me organize my thinking", 
)
response

In [ ]:
response.response